In [44]:
import os
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

n_glyphs = 6494

df1 = pd.read_csv('/Users/IMYV/Desktop/cfdb-django/glyphs-aligned-w-std_sign-images.csv', usecols=['std_sign', 'glyph_img'])
df2 = pd.read_csv('/Users/IMYV/Desktop/cfdb-django/glyph_count.csv', usecols=['std_sign', 'glyph_img']).sort_values(by='glyph_img')

f=open('/Users/IMYV/Desktop/cfdb-django/media/training_set/trainig_data.csv', 'a+')

df_group = df1.groupby(by=['std_sign'])
df_group = sorted(df_group, key=lambda x: len(x[1])) #https://stackoverflow.com/questions/22291395/sorting-the-grouped-data-as-per-group-size-in-pandas

for name,group in df_group:
    if  group.count()[0] > 50:
        group.reset_index().to_csv(f, header=None)
        
df_usable = pd.read_csv('/Users/IMYV/Desktop/cfdb-django/media/training_set/trainig_data.csv')
df_shuffled = df_usable.sample(frac=1).reset_index(drop=True)
df_shuffled.to_csv('/Users/IMYV/Desktop/cfdb-django/media/training_set/trainig_data.csv')

df_example: 